- Image scale normalization
- blurring ✅
- shifting
- merging?

In [228]:
import matplotlib.image as mimg
import matplotlib.pyplot as plt
import numpy as np
import math

In [229]:
img1 = mimg.imread('Input Images/Spiral.jpg')
img2 = mimg.imread('Input Images/Oogway_forest.jpg')

In [230]:
# other functions

# overlay dependency
def avg_pixel_weighted(channel_val1, weight1, channel_val2, weight2):
    adjusted_weight_total = weight1 + weight2
    return ( (channel_val1 * weight1 + channel_val2 * weight2)/adjusted_weight_total)


# blur dependency
def avg_color_grid(img, blur_size, start_pixel):

    r_total, g_total, b_total=0, 0, 0
    x_dim, y_dim = start_pixel
    counter = blur_size**2

    for i in range(blur_size):
        for j in range(blur_size):

            r_total += img.item((x_dim + i, y_dim + j, 0))
            g_total += img.item((x_dim + i, y_dim + j, 1))
            b_total += img.item((x_dim + i, y_dim + j, 2))

    return np.array((r_total / counter, g_total / counter, b_total / counter))


In [231]:
# 2 image processing

def overlay(img1, img2):
    IMG1_WEIGHT = 1
    IMG2_WEIGHT = 1
    
    max_x_size = max(img1.shape[0], img2.shape[0])
    max_y_size = max(img1.shape[1], img2.shape[1])

    new_img_arr = np.empty((max_x_size, max_y_size, 3))

    for i in range(max_x_size):
        for j in range(max_y_size):
            for k in range(3):

                new_img_arr[i, j, k] = avg_pixel_weighted(img1.item((i, j, k)), IMG1_WEIGHT, img2.item((i, j, k)), IMG2_WEIGHT) / 255  # 4 sec

    return new_img_arr


In [370]:
def mosaic(img):
    # blur_size = int(input("Blur size: "))
    pixelate_amount = 25
    # 2 - 17.8s
    # 3 - 24.6s
    # 5 - ~47s
    # 10 - 2m 37.5s

    height = img.shape[0]
    width = img.shape[1]

    new_img_arr = np.empty((height, width, 3))

    for i in range(0, height - pixelate_amount, pixelate_amount):
        for j in range(0, width - pixelate_amount, pixelate_amount):
            color = avg_color_grid(img, pixelate_amount, (i, j)) / 255
            
            for _i in range(pixelate_amount):
                for _j in range(pixelate_amount):
                    new_img_arr[i + _i, j + _j] = color

    return new_img_arr


In [360]:
# BLUR

def blur(img):
    # blur_size = int(input("Blur size: "))
    blur_size = 5
    # 2 - 17.8s
    # 3 - 24.6s
    # 5 - ~47s
    # 10 - 2m 37.5s

    height = img.shape[0]
    width = img.shape[1]

    new_img_arr = np.empty((height, width, 3))

    for i in range(height - blur_size):
        for j in range(width - blur_size):
            new_img_arr[i, j] = avg_color_grid(img, blur_size, (i, j)) / 255

    return new_img_arr


In [384]:
# SHIFTING
import random

def shift(img):
    # operation = int(input("what operation should be performed? (0:linear, 1:sin, 2:cos, 3:tan, 4:random)  "))
    operation = 1
    multiplicant = 0

    start_val = 0
    START_OFFSET = 0
    SHIFT_CONSTANT_X = 15
    SHIFT_CONSTANT_Y = 15

    height = img.shape[0]
    width = img.shape[1]

    new_img_arr = np.empty((height, width, 3))

    for i in range(height):
        for j in range(width):
            for k in range(3):
                if operation == 4:
                    multiplicant = random.random() * 2 - 1  

                new_img_arr[i, j, k] = img.item((
                    ((int(multiplicant * SHIFT_CONSTANT_X)) + i + START_OFFSET) %
                    height,
                    ((int(multiplicant * SHIFT_CONSTANT_Y)) + j + START_OFFSET) %
                    width, k)) / 255

        multiplicant = start_val if operation == 0 else math.sin(start_val) if operation == 1 else math.cos(start_val) if operation == 2 else math.tan(start_val) if operation == 3 else 0

        start_val+=1.6 # for sin or cos, min and max shift is from 0 to 1.5707 (π/2)

    return new_img_arr


In [385]:
# plt.imshow(shift(img2))
# plt.imsave("Output Images/Distort/Sin.jpg", shift(img2))
plt.imsave("Output Images/Distort/delete.jpg", shift(img2))